<a href="https://colab.research.google.com/github/JoaoGuilhermeTP/assistentedehomologacao/blob/main/Homologa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-docx

In [ ]:
#IMPORTING MODULES
import os, re, docx, pandas as pd
pd.options.display.max_colwidth = 1000
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
from datetime import datetime,timezone, timedelta
from google.colab import drive
drive.mount('/content/drive')

###############################################################################################################################################################
#DATAFRAMES
df_ferias = pd.read_excel('/content/drive/MyDrive/Work/férias.xlsx') #vacations
df = pd.read_excel('/content/drive/MyDrive/Work/competencias.xlsx',index_col='sigla') #competencies
###############################################################################################################################################################
#DEALING WITH DATES

#today
timezone_offset = -3.0 # Brazil time zone
tzinfo = timezone(timedelta(hours = timezone_offset))
today = datetime.now(tzinfo)
todayAsString = today.strftime('Em %d de %B de %Y.')

#tomorrow
if today.weekday() == 4:
  tomorrow = today + timedelta(days=3)
else:
  tomorrow = today + timedelta(days=1)
tomorrow = tomorrow.strftime('Em %d de %B de %Y.')

months = {'January':'janeiro','February':'fevereiro','March':'março','April':'abril','May':'maio','June':'junho','July':'julho','August':'agosto','September':'setembro','October':'outubro','November':'novembro','December':'dezembro'}
for month in months:
    if month in todayAsString:
        todayAsString = todayAsString.replace(month,months[month])
    if month in tomorrow:
        tomorrow = tomorrow.replace(month,months[month])
###############################################################################################################################################################
# CUSTOM FUNCTIONS

#Signatures
def assinaturas():
    filt = df.index.isin(secretarias) # Filtering by secretary
    if len(df[filt]['titulares'].unique()) % 2 == 0: 
      qt_linhas = int(len(df[filt]['titulares'].unique()) / 2)
    else:
      qt_linhas = int((len(df[filt]['titulares'].unique()) + 1) / 2)
    if len(df[filt]['titulares'].unique()) > 1:
      table = doc.add_table(qt_linhas, cols=2)
    else:
      table = doc.add_table(qt_linhas, cols=1)
    i = 0
    for row in table.rows:
      for cell in row.cells:
        try:
          filt_2 = df['titulares'] == df[filt]['titulares'].unique()[i] # Filtrando por secretário
          cell.text = f"\n\n{df[filt]['titulares'].unique()[i]}\n"
          if len(df[filt_2]['assinatura']) > 1:
            cell.text = cell.text + ', '.join(df[filt][filt_2]['assinatura'][0:-1])
            cell.text = cell.text + ' e ' + df[filt_2]['assinatura'][-1]
          elif len(df[filt_2]['assinatura']) < 2:
            cell.text = cell.text + df[filt_2]['assinatura'].to_string(index=False)
          cell.text = cell.text.replace("'","").replace('sigla\n','')
          p = cell.paragraphs[0]
          run = cell.paragraphs[0].runs[0]
          run.font.bold = True
          p.alignment = WD_ALIGN_PARAGRAPH.CENTER
          i = i + 1
        except: pass

#Vacations
def ferias(inicio,fim,secretaria,responsavel,enderecamento,assinatura):
    responsavel = responsavel.upper()
    if today >= inicio and today <= fim:
        df.loc[secretaria,'titulares'] = responsavel
        df.loc[secretaria,'endereçamento'] = enderecamento
        df.loc[secretaria,'assinatura'] = assinatura

for i in list(df_ferias.index):
  ferias(df_ferias.loc[i,'inicio'],df_ferias.loc[i,'fim'],df_ferias.loc[i,'secretaria'],df_ferias.loc[i,'responsavel'],df_ferias.loc[i,'enderecamento'],df_ferias.loc[i,'assinatura'])

###############################################################################################################################################################
#PREGOEIRO
pregoeiros = {
    'nome' : ['EDNEY GARCIA CAMARA','FABIANO BALLIANO MALAVASI','ROBERTO WEGE FONSECA','IZABEL LUIZA PARISI ZAMPIERI'],
    'cargo_pregoeiro' : ['Pregoeiro','Pregoeiro','Pregoeiro','Pregoeira'],
    'subscritor_pregoeiro' : ['este subscritor','este subscritor','este subscritor','esta subscritora'],
    'endereçamento' : ['Sr. Pregoeiro','Sr. Pregoeiro','Sr. Pregoeiro','Sra. Pregoeira']
}

opcao = int(input('''Quem foi o(a) pregoeiro(a)?

1 - EDNEY GARCIA CAMARA
2 - FABIANO BALLIANO MALAVASI
3 - ROBERTO WEGE FONSECA
4 - IZABEL LUIZA PARISI ZAMPIERI\n''')) - 1

pregoeiro = pregoeiros['nome'][opcao]
cargo_pregoeiro = pregoeiros['cargo_pregoeiro'][opcao]
subscritor_pregoeiro = pregoeiros['subscritor_pregoeiro'][opcao]
endereçamento_pregoeiro = pregoeiros['endereçamento'][opcao]
###############################################################################################################################################################
#CONSIDER DECISORY ACTS
considerar_atos = None 
while considerar_atos != 'S' and considerar_atos != 'N':
    considerar_atos = input('Deseja considerar os atos decisórios? (S/N) ').upper()
    if considerar_atos != 'S' and considerar_atos != 'N':
        print('\nDigite apenas "S" para Sim, ou "N" para Não\n')

###############################################################################################################################################################
#DEFINING PARTICIPANT SECRETARIES
temp_secretarias = str(input('Informe as secretarias participantes, incicianto pela gestora: ')).upper().split()
i = 0
for x in temp_secretarias:
  while x not in df.index:
    correcao = input(f'A secretaria {x} não foi encontrada. Deseja informar novamente? S/N').upper()
    while correcao != 'S' and correcao != 'N':
      correcao = input('Resposta inválida. Por favor, entre com apenas <S> para Sim ou <N> para Não.')
    if correcao == 'S':
      x_sub = input('Informe a secretaria correta: ').upper()
      x = x_sub
      temp_secretarias[i] = x
    else:
      del temp_secretarias[i]
      break
  i = i + 1
secretaria_gestora = temp_secretarias[0]
temp_secretarias = sorted([(float(x.split('-')[1]),x.split('-')[0]) for x in temp_secretarias])
temp_secretarias = [str(x[1]) + '-' + str(x[0]) for x in temp_secretarias]

secretarias = []
for x in temp_secretarias:
  x_sigla, x_numero = x.split('-')[0], str(x.split('-')[1])
  x_numero = x_numero.split('.')
  if x_numero[1] == '0':
    x_numero = x_numero[0]
  else:
    x_numero = '.'.join(x_numero)
  x = f'{x_sigla}-{x_numero}'
  secretarias.append(x)

if len(secretarias) > 1:
    string_secretarias = ', '.join(secretarias[:-1])
    string_secretarias = string_secretarias + ' e ' + secretarias[-1]
else:
    string_secretarias = secretarias[0]
###############################################################################################################################################################
#JUDGEMENT CRITERIA
criterio = input('Informe o critério de julgamento: ').upper()

###############################################################################################################################################################
# LAWS STRING
df_filt = df[df.index.isin(secretarias)]
leis = set()
for x in df_filt['lei'].unique():
    leis.add(x)

for x in leis:
    d = {}
    artigos = []
    cc1 = set()
    cc2 = set()
    filt = df_filt['lei'] == x
    string = ''
    for artigo in df_filt.loc[filt,'artigo']:
        artigos.append(artigo)
    for i in df_filt.loc[filt,'cc1']:
        if i != None and i != ' ' and i != '':
            cc1.add(i)
    for i in df_filt.loc[filt,'cc2']:
        if i != None and i != ' ' and i != '' and i not in cc1:
            cc2.add(i)
    cc1.union(cc2)
    d[x] = artigos
    d['cc'] = cc1
    if len(artigos) > 1:
        text = '; '.join(d[x][:-1])
        text = text + ' e ' + d[x][-1]
    else:
        text = ''.join(artigos)
    text = text + ' da Lei Complementar nº. ' + x 
    if len(d['cc']) > 0:
        print(d['cc'])
        text = text + ', c.c.'
        cc = ', c.c. '.join(d['cc'])
        text = text + cc
    cc=''
###############################################################################################################################################################
# CREATING ADRESSINGS STRING
enderecamentos = []
for enderecamento in df_filt['endereçamento']:
    enderecamentos.append(enderecamento)

possiveis_enderecamentos = []
for enderecamento in df['endereçamento'].unique():
    possiveis_enderecamentos.append(enderecamento)

possiveis_enderecamentos_plurais = []
for enderecamento in possiveis_enderecamentos:
    enderecamento = enderecamento.replace('Sr. ','Srs. ').replace('Sra. ','Sras. ').replace('Secretário','Secretários').replace('Secretária','Secretárias').replace('Subsecretário','Subsecretários').replace('Subsecretária','Subsecretárias').replace('Municipal','Municipais').replace('Secretaria','Secretarias').replace('Interino','Interinos').replace('Interina','Interinas').replace('Substituto','Substitutos').replace('Substituta','Substitutas')
    possiveis_enderecamentos_plurais.append(enderecamento)

enderecamentos_dict = dict(zip(possiveis_enderecamentos, possiveis_enderecamentos_plurais))

enderecamento_final = set()

for x,y in enderecamentos_dict.items():
    count = 0
    for enderecamento in enderecamentos:
        if x == enderecamento:
            count = count + 1
    if count == 1:
        enderecamento_final.add(x)
    elif count > 1:
        enderecamento_final.add(y)
    else: pass

enderecamento_final = list(enderecamento_final)

if len(enderecamento_final) > 1:
    string_enderecamento = ', '.join(enderecamento_final[:-1])
    string_enderecamento = string_enderecamento + ' e ' + enderecamento_final[-1]
else:
    string_enderecamento = enderecamento_final[0]
###############################################################################################################################################################
# READING THE MINUTES FILES
tabela = open('tabela.csv','w')
campos = ['OC','item_oc','descrição','quantidade','valor','empresa','CNPJ','marca','resultado_ata','decisao_pregoeiro','decisao_autoridade']
print('Item','OC','Item OC','QTDE','Valor','Empresa','CNPJ','Marca','Resultado','Descrição',sep='#',file=tabela)

atas = []
for filename in os.listdir(r'/content'):
  if filename.endswith('.txt') and filename.startswith('ata'):
    atas.append(filename)
    atas = sorted(atas)
d = {}
ocs_completas = []
numero = 0
marker = 'negative'

for filename in atas:
  for line in open(filename,encoding='utf8'):
    line = line.rstrip()
    if line.startswith('Número da OC'):
      oc = line.split()[3][-3:]
      ocs_completas.append(line.split()[3])
    if line.startswith('Pregão Eletrônico nº :'):
      titulo = ' '.join(line.split(' : ')).upper()
    if line.startswith('Processo nº :'):
      processo = line.split(' : ')[1]
    if line.startswith('Objeto : '):
      objeto = line.split(' : ')[1]
    if line.startswith('ITEM'):
      dados = {}
      for x in campos:
        dados[x] = '-'
        dados['OC'] = oc
      numero = numero+1
      d[numero] = dados
      dados['ata'] = filename.split('.')[0]
      dados['item_oc'] = line.split()[1]
    if line.startswith('Descrição: '):
        descrição = line.split(': ')[1]
        dados['descrição'] = (descrição)
    if line.startswith('Quantidade / Unidade de Fornecimento:'):
      dados['quantidade'] = (re.search('[0-9.,]+',line)).group()
    if line.startswith('Menor Valor:'):
      try:
        dados['valor'] = (re.search('[0-9.,]+',line)).group()
      except: pass
    if line.startswith('CNPJ/CPF - Vencedor:'):
      try:
        dados['empresa'] = (line.split(': ')[1]).split(' - ')[1]
        cnpj = (line.split(': ')[1]).split(' - ')[0]
        dados['CNPJ'] = cnpj[0:2]+'.'+cnpj[2:5]+'.'+cnpj[5:8]+'/'+cnpj[8:12]+'-'+cnpj[12:]
      except: pass
    if line.startswith('Resultado do Item:'):
      dados['resultado_ata'] = line.split(': ')[1].upper()
    if line.startswith('Licitante'):
      if 'Marca' in line:
        marker = 'positive'
    try:
      if line.startswith(dados.get('empresa')) and marker == 'positive':
        dados['marca'] = line.split('\t')[2]
        marker = 'negative'
    except: pass

for x,y in d.items():
  print(str(x),y.get('OC'),y.get('item_oc'),y.get('quantidade'),y.get('valor'),y.get('empresa').upper(),y.get('CNPJ'),y.get('marca').upper(),y.get('resultado_ata'),y.get('descrição'),sep='#',file=tabela)

tabela.close()
data =  pd.read_csv(r'/content/tabela.csv',sep='#', engine='python')
data.set_index('Item',inplace=True)

os.remove('tabela.csv')
###############################################################################################################################################################
#READING THE DECISORY ACTS FILES
if considerar_atos.upper() == 'S':
  atos = []
  for filename in os.listdir(r'/content'):
    if filename.endswith('.txt') and filename.startswith('ato'):
      atos.append(filename)
      atos = sorted(atos)

  numero = 0
  marker = 'negative'
  resultados = 'DESERTO REVOGADO CANCELADO FRACASSADO ANULADO HOMOLOGADO ADJUDICADO'.split()

  for filename in atos:
    for line in open(filename,encoding='utf8'):
      line = line.rstrip()
      if line.startswith('Número da OC'):
        oc = line.split()[3][-3:]
      if line.startswith('ato decisorio'):
        decisoes = []
        marker = 'negative'
        numero = numero + 1
      for resultado in resultados:
        if line == resultado:
          data.loc[numero,'Resultado'] = resultado
###############################################################################################################################################################
#SAVING THE EXCEL FILE
data.to_excel(r'/content/tabela.xlsx', index = True)

###############################################################################################################################################################
# INPUTS ABOUT DOCUMENTS RELATED TO THE PROCUREMENT
string_documentos = []

texto_atas = ''
for oc in ocs_completas:
  texto_atas = f'Ata de Sessão Pública às fls. #, referente à Oferta de Compra nº. {oc}'
  folhas = input(f'Em quais folhas está juntada a Ata de Sessão referente à OC {oc}? ')
  texto_atas = texto_atas.replace('#',folhas)
  string_documentos.append(texto_atas)

verba_federal = input('Os documentos foram publicados no Diário Oficial da União? (S/N)').upper()

if verba_federal == 'S':
  texto_jornais = 'nos jornais "Diário Oficial do Estado de São Paulo" e "Diário Oficial da União" em @, conforme comprovantes juntados sob folhas #, '
else:
  texto_jornais = 'no jornal "Diário Oficial do Estado de São Paulo" em @, conforme comprovante juntado sob folha #, '

adicoes = []
while True:
    adicao = input('''
    O que você gostaria de adicionar? (Pressione ENTER caso não deseje adicionar nada: )

    1 - Recurso Administrativo
    2 - Resultado de Análise das Amostras
    3 - Resultado de Análise da Documentação
    4 - Ata de Sessão Pública Complementar
    5 - Outro
    ''')
    if adicao == '1':
        recursante = input('Qual é a empresa que apresentou este recurso?').upper()
        processo_recurso = input('Qual é o processo administrativo deste recurso? ')
        recurso = f'Despacho referente ao Recurso Administrativo interposto pela empresa {recursante}, publicado {texto_jornais}do Processo Administrativo nº. {processo_recurso}'
        folhas = input(f'Em quais folhas estão juntados os comprovantes de publicação? ')
        recurso = recurso.replace('#',folhas)
        datas = input(f'Quando foi feita a publicação nos jornais? ')
        recurso = recurso.replace('@',datas)
        string_documentos.append(recurso)
    elif adicao == '2':
        resultado_amostra = f'Resultado de Análise das Amostras, publicado {texto_jornais}'
        folhas = input(f'Em quais folhas está juntado o comprovantes de publicação? ')
        resultado_amostra = resultado_amostra.replace('#',folhas)
        datas = input(f'Quando foi feita a publicação nos jornais? ')
        resultado_amostra = resultado_amostra.replace('@',datas)
        string_documentos.append(resultado_amostra)
    elif adicao == '3':
        resultado_documentacao = f'Resultado de Análise da Documentação, publicado {texto_jornais}'
        folhas = input(f'Em quais folhas está juntado o comprovantes de publicação? ')
        resultado_documentacao = resultado_documentacao.replace('#',folhas)
        datas = input(f'Quando foi feita a publicação nos jornais? ')
        resultado_documentacao = resultado_documentacao.replace('@',datas)
        string_documentos.append(resultado_documentacao)
    elif adicao == '4':
        oc_encontrada = False
        while oc_encontrada == False:
            if len(ocs_completas) < 2:
                oc = ocs_completas[0]
                oc_encontrada = True
            else:
                oc_complementar = input('Quais são os três últimos números da OC referente a esta Ata Complementar?')
                for oc in ocs_completas:
                    if oc[-3:] == oc_complementar:
                        oc_complementar = oc
                        oc_encontrada = True  
            ata_complementar = f'Ata de Sessão Complementar referente à Oferta de Compra nº. {oc}, juntada sob folha #'
            folhas = input(f'Em quais folhas está juntada a Ata de Sessão Complementar referente à OC {oc}? ')
            ata_complementar = ata_complementar.replace('#',folhas)
            string_documentos.append(ata_complementar)
            if oc_encontrada == True:    
                break
            else:
                print('--------- POR FAVOR, VERIFIQUE O NÚMERO DA OC')
    elif adicao == '5':
        temp = input('O que você gostria de adicionar? ')
        outro = f'{temp}, publicado {texto_jornais}'
        string_documentos.append(outro)
    else:
        break
    adicoes.append(adicao)

texto_atas_final = ', '.join(string_documentos[:-1])
texto_atas_final = texto_atas_final + ' e ' + string_documentos[-1] +' e Atos Decisórios juntados sob folhas # do Processo Administrativo em epígrafe'
folhas = input(f'Em quais folhas estão juntados os Atos Decisórios ')
texto_atas_final = texto_atas_final.replace('#',folhas)
###############################################################################################################################################################
doc = docx.Document(f'/content/drive/MyDrive/Work/modelo - TERMO DE HOMOLOGAÇÃO.docx')
###############################################################################################################################################################

# CREATING PREAMBLE STRING
if len(ocs_completas) < 2:
  texto_ocs = 'NÚMERO DA OFERTA DE COMPRA: '+ ocs_completas[0]
else:
  texto_ocs = 'NÚMERO DAS OFERTAS DE COMPRAS: '
  for oc in ocs_completas:
      texto_ocs = texto_ocs + '\n' + oc
preambulo = f'''TERMO DE HOMOLOGAÇÃO
{titulo}
PROCESSO ADMINISTRATIVO Nº. {processo}
OBJETO: "{objeto}"
{texto_ocs}\n'''
###############################################################################################################################################################
# 1st COTA:

nomes = 'João Guilherme Tasca Pereira # Josicleide das Neves Chagas # Thais de Oliveira Varjão # Luiz Cubilia Neto'.split(' # ')
meu_nome = input('Digite o seu primeiro nome: ').capitalize()
if meu_nome == 'João':
  end = 'Sr. João Guilherme'
elif meu_nome == 'Josicleide':
  end = 'Sra. Josicleide'
elif meu_nome == 'Thais':
  end = 'Sra. Thais'
elif meu_nome == 'Luiz':
  end = 'Sr. Luiz Cubilia Neto'

for nome in nomes:
  if meu_nome in nome:
    meu_nome = nome

p = doc.add_paragraph()
p.add_run(f'À\nSEAD-5.2.2\n{end}').bold = True

p = doc.add_paragraph()
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
p.add_run('\n\tSegue o presente para regularização no ')
r = p.add_run(f'SIAM')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = True
r = p.add_run(f', consoante aos demais procedimentos administrativos.')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False

doc.add_paragraph(f'\n{todayAsString}')
p = doc.add_paragraph()
p.add_run(f"""

{pregoeiro}
{cargo_pregoeiro}""").bold = True
p.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_page_break()
###############################################################################################################################################################

# 2nd COTA
p = doc.add_paragraph()
p.add_run(f'À\nSEAD-5.2.2\n{endereçamento_pregoeiro}').bold = True

p = doc.add_paragraph()
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
p.add_run('\n\tApós regularização no ')
r = p.add_run(f'SIAM')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = True
r = p.add_run(f', conforme comprovante anexo aos autos, segue para ciência, conferência e demais determinações de Vossa Senhoria.')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False

doc.add_paragraph(f'\n{todayAsString}')
p = doc.add_paragraph()
p.add_run(f"""
{meu_nome.upper()}
Departamento de Licitações""").bold = True
p.alignment = WD_ALIGN_PARAGRAPH.CENTER
###############################################################################################################################################################

# COTA ESPECÍFICA SEAD
if secretaria_gestora == 'SEAD-5':
    p = doc.add_paragraph()
    p.add_run(f'\nÀ\nSEAD-52\nSra. Subsecretária de Gestão de Meios').bold = True

    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    p.add_run('\n\tCiente quanto às providências relativas ao ')
    r = p.add_run(f'{titulo}, ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = True
    r = p.add_run(f'após regularização no SIAM, conforme cota supra e demais atos praticados conforme ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False
    r = p.add_run(f'{texto_atas_final}, bem como os valores negociados por {subscritor_pregoeiro}, segue para ciência de ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

    if len(secretarias) > 1:
        r = p.add_run('Vossas Senhorias ')
        font = r.font
        font.name = 'Times New Roman'
        font.size = Pt(12)
        r.bold = False
    else:
        r = p.add_run('Vossa Senhoria ')
        font = r.font
        font.name = 'Times New Roman'
        font.size = Pt(12)
        r.bold = False

    item_homologacao = input('Qual o item da homologação no edital? ')
    item_recuperacao = input('Qual o subitem sobre a situação de Recuperação Extrajducial? ')

    r = p.add_run(f'de todo o procedimento licitatório e demais determinações quanto à homologação do certame ou não, conforme disposto no item {item_homologacao} e seus subitens do Edital.')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False
    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    r = p.add_run(f'\tInformo que a empresa não se enquadra na situação de recuperação judicial e/ou extrajudicial, dispostas no subitem {item_recuperacao} do Edital.')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

    doc.add_paragraph(f'\n{todayAsString}')
    p = doc.add_paragraph()
    p.add_run(f"""
    {pregoeiro}
    {cargo_pregoeiro}""").bold = True
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER

# 4ª COTA CASO SEAD SEJA GESTORA
    p = doc.add_paragraph()
    p.add_run(f'\nÀ\n{string_secretarias}\n{string_enderecamento}').bold = True

    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    r = p.add_run(f'\n\tSegue para ciência de ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

    if len(secretarias) > 1:
        r = p.add_run('Vossas Senhorias ')
        font = r.font
        font.name = 'Times New Roman'
        font.size = Pt(12)
        r.bold = False
    else:
        r = p.add_run('Vossa Senhoria ')
        font = r.font
        font.name = 'Times New Roman'
        font.size = Pt(12)
        r.bold = False

    r = p.add_run(f'''de todo o procedimento licitatório e demais determinações quanto à homologação do certame ou não, conforme disposto no item {item_homologacao} e seus subitens do Edital.''')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

    doc.add_paragraph(f'\n{todayAsString}')
    p = doc.add_paragraph()
    p.add_run(f"""
    CARLA BATISTA DE SOUZA
    Subsecretária de Gestão de Meios""").bold = True
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()
###############################################################################################################################################################

# 3rd COTA
if secretaria_gestora != 'SEAD-5':
    p = doc.add_paragraph()
    p.add_run(f'\nÀ\n{string_secretarias}\n{string_enderecamento}').bold = True

    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    p.add_run('\n\tCiente quanto às providências relativas ao ')
    r = p.add_run(f'{titulo}, ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = True
    r = p.add_run(f'após regularização no SIAM, conforme cota supra e demais atos praticados conforme ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False
    r = p.add_run(f'{texto_atas_final}, bem como os valores negociados por {subscritor_pregoeiro}, segue para ciência de ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

    if len(secretarias) > 1:
        r = p.add_run('Vossas Senhorias ')
        font = r.font
        font.name = 'Times New Roman'
        font.size = Pt(12)
        r.bold = False
    else:
        r = p.add_run('Vossa Senhoria ')
        font = r.font
        font.name = 'Times New Roman'
        font.size = Pt(12)
        r.bold = False

    item_homologacao = input('Qual o item da homologação no edital? ')
    item_recuperacao = input('Qual o subitem sobre a situação de Recuperação Extrajducial? ')

    r = p.add_run(f'de todo o procedimento licitatório e demais determinações quanto à homologação do certame ou não, conforme disposto no item {item_homologacao} e seus subitens do Edital.')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    r = p.add_run(f'\tInformo que a empresa não se enquadra na situação de recuperação judicial e/ou extrajudicial, dispostas no subitem {item_recuperacao} do Edital.')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

    doc.add_paragraph(f'\n{todayAsString}')
    p = doc.add_paragraph()
    p.add_run(f"""

    {pregoeiro}
    {cargo_pregoeiro}""").bold = True
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()
###############################################################################################################################################################
# INSERTING THE PUBLICATION QUOTE
registro_de_preços = input('Será gerada ata de registro de preços? (S/N)').upper()
contrato = input('Será gerado Contrato? (S/N)').upper()

total_de_empresas = len(data['Empresa'].unique())
texto_jornais = texto_jornais.split(' em')[0]

p = doc.add_paragraph()
p.add_run(f'\nÀ\nSEAD-5.2.1.1\nSr. Diretora').bold = True

p = doc.add_paragraph()
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

if len(secretarias) > 1:
    r = p.add_run(f'\n\tSolicitamos publicar o Termo de Homologação, conforme consta abaixo, {texto_jornais}')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False
else:
    r = p.add_run(f'\n\tSolicito publicar o Termo de Homologação, conforme consta abaixo, {texto_jornais}')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

compras_dict = {
    'SESURB-15':'SESURB-15.1.1.3',
    'SEDUC-9':'SEDUC-9.1.1.6',
    'SESAP-10':'SESAP-10.6.1.2',
    'SEEL-20':'SEEL-20.1.1.1.3',
    'SEAS-8':'SEAD-8.3.2',
    'SETRAN-16':'SETRAN-16.2.2'
}

if secretaria_gestora in compras_dict:
    compras = compras_dict[secretaria_gestora]
else:
    compras = 'SEAD-5.2.2.1'

r = p.add_run(f', com posterior encaminhamento à ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False

r = p.add_run(compras)
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = True

r = p.add_run(' para disponibilização deste no site www.praiagrande.sp.gov.br, bem como inserção de dados na ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False

r = p.add_run('AUDESP - FASE IV (Licitações e Contratos Administrativos). ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = True

if contrato == 'S' or registro_de_preços == 'S':
    r = p.add_run('Após, encaminhar à ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False
    r = p.add_run('SEAD-5.2.1.1 ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = True
    if contrato == 'S' and registro_de_preços == 'S':
        if total_de_empresas > 1:
            r = p.add_run('para elaboração dos Termos de Ata e Contratos.')
            font = r.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
            r.bold = False
        else:
            r = p.add_run('para elaboração do Termo de Ata e Contrato.')
            font = r.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
            r.bold = False
    elif contrato == 'S' and registro_de_preços == 'N':
        if total_de_empresas > 1:
            r = p.add_run('para elaboração dos Contratos.')
            font = r.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
            r.bold = False
        else:
            r = p.add_run('para elaboração do Contrato.')
            font = r.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
            r.bold = False
    elif contrato == 'N' and registro_de_preços == 'S':
        if total_de_empresas > 1:
            r = p.add_run('para elaboração dos Termos de Ata.')
            font = r.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
            r.bold = False
        else:
            r = p.add_run('para elaboração do Termo de Ata.')
            font = r.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
            r.bold = False

# INSERTING THE PREAMBLE PARAGRAPH
p = doc.add_paragraph()
p = doc.add_paragraph()
p.alignment = WD_ALIGN_PARAGRAPH.LEFT
r = p.add_run(f'{preambulo}')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = True
###############################################################################################################################################################

# HOMOLOGATION PARAGRAPH
if registro_de_preços == 'S':
    classifica_adjudica = 'CLASSIFICARAM'
else:
    if '1' in adicoes:
      if len(secretarias) > 1:
        classifica_adjudica = 'ADJUDICAMOS'
      else:
        classifica_adjudica = 'ADJUDICO'
    else:
      classifica_adjudica = 'ADJUDICARAM'

if 'LOTE' in criterio:
    if len(data) > 1:
        item_lote = 'dos respectivos lotes'
    else:
        item_lote = 'do lote lote'
else:
    if len(data) > 1:
        item_lote = 'dos respectivos itens'
    else:
        item_lote = 'do item'

p = doc.add_paragraph()
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
r = p.add_run(f'\tConsiderando as decisões exaradas em {texto_atas_final}, que ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False
r = p.add_run(f'{classifica_adjudica} ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = True

crase = 'às'
if total_de_empresas > 1:
    if classifica_adjudica == 'ADJUDICO' or classifica_adjudica == 'ADJUDICAMOS':
      crase = 'às'
    elif classifica_adjudica == 'CLASSIFICARAM':
      crase = 'as'
    empresa = ''
    if 'LOTE' in criterio:
        item_lote = 'dos respectivos lotes'
        if classifica_adjudica == 'ADJUDICARAM':
          string_empresas = f'{crase} empresas abaixo relacionadas, em primeiro lugar, para o fornecimento {item_lote}' 
    else: 
        item_lote = 'dos respectivos itens'
        if classifica_adjudica == 'ADJUDICARAM':
          string_empresas = f'{crase} empresas abaixo relacionadas, em primeiro lugar, para o fornecimento {item_lote}'
    r = p.add_run(f'{crase} empresas abaixo relacionadas, em primeiro lugar, para o fornecimento {item_lote}, em razão do ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False

else:
    empresa = data['Empresa'].unique()[0]
    if classifica_adjudica == 'ADJUDICO' or classifica_adjudica == 'ADJUDICAMOS':
      crase = 'à'
    elif classifica_adjudica == 'CLASSIFICARAM':
      crase = 'a'
    if 'LOTE' in criterio:
        if len(data) > 1:
            item_lote = 'dos lotes'
            string_empresas = f'{crase} empresa {empresa}, em primeiro lugar, para o fornecimento {item_lote}'
        else:
            item_lote = 'do lote'
            string_empresas = f'{crase} empresa {empresa}, em primeiro lugar, para o fornecimento {item_lote}'
    else:
        if len(data) > 1:
            crase = 'às'
            item_lote = 'dos itens'
            string_empresas = f'{crase} empresa {empresa}, em primeiro lugar, para o fornecimento {item_lote}'
        else:
            crase = 'à'
            item_lote = 'do item'
            string_empresas = f'{crase} empresa {empresa}, em primeiro lugar, para o fornecimento {item_lote}'
    r = p.add_run(f'{crase} empresa ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False
    r = p.add_run(empresa.upper())
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = True
    r = p.add_run(f', em primeiro lugar, para o fornecimento {item_lote}, em razão do ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = False  

r = p.add_run(f'{criterio.upper()}, ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = True
r = p.add_run(f'sendo condição mais vantajosa para a Administração, ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False

if len(secretarias) > 1:
    r = p.add_run('HOMOLOGAMOS ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = True
else:
    r = p.add_run('HOMOLOGO ')
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    r.bold = True

r = p.add_run('a presente licitação, nos termos do ')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False
r = p.add_run(f'{text}  e nos termos do Art. 5º, inciso IV do Decreto Municipal nº. 3.593/2003.')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False
r = p.add_run('')
font = r.font
font.name = 'Times New Roman'
font.size = Pt(12)
r.bold = False
###############################################################################################################################################################

# IMPRIMINDO AS EMPRESAS E SEUS RESPECTIVOS ITENS
if total_de_empresas > 1:
    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.LEFT
    for oc in ocs_completas:
        if len(ocs_completas) > 1:
            p = doc.add_paragraph()
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER
            r = p.add_run(f'\nOferta de Compra nº. {oc}')
            p = doc.add_paragraph()
            r.bold = True
            font = r.font
            font.name = 'Times New Roman'
            font.size = Pt(12)
        if considerar_atos.upper() == 'S':
            filtro = (data['Resultado'] == 'HOMOLOGADO') & (data['OC'] == int(oc[-3:]))
        else:
            filtro = data['OC'] == int(oc[-3:])
        new_data = data[filtro]
        for empresa in new_data['Empresa'].unique():
            if empresa != '-':
                filt = (new_data['Empresa'] == empresa)
                itens = []
                for item in new_data[filt].index:
                    itens.append(str(item))
                if len(itens) > 1:
                    itens_vencedores = 'itens '
                    itens_vencedores = itens_vencedores + ', '.join(itens[:-1])
                    itens_vencedores = itens_vencedores + ' e ' + itens[-1]
                else:
                    itens_vencedores = 'item '
                    itens_vencedores = itens_vencedores + itens[0]
                p = doc.add_paragraph()
                p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
                r = p.add_run(f'{empresa}: ')
                r.bold = True
                font = r.font
                font.name = 'Times New Roman'
                font.size = Pt(12)
                r = p.add_run(f'{itens_vencedores}')
                r.bold = False
                font = r.font
                font.name = 'Times New Roman'
                font.size = Pt(12)

        # IMPRINIDNO ITENS SEM VENCEDORES
        p = doc.add_paragraph()
        if considerar_atos.upper() == 'S':
            for resultado in data['Resultado'].unique():
                string_itens = ''
                if resultado != 'HOMOLOGADO' and resultado != '-':
                    filt = (data['Resultado'] == resultado) & (data['OC'] == int(oc[-3:]))
                    itens = []
                    if len(data[filt]) > 0:
                        for item in data[filt].index:
                            itens.append(str(item))
                        if len(itens) > 1:
                            string_itens = '- Os itens '
                            string_itens = string_itens + ', '.join(itens[:-1])
                            string_itens = string_itens + ' e ' + itens[-1]
                            string_itens = string_itens + ' foram declarados ' + resultado.lower() + 's'
                        else:
                            resultado = resultado.lower()
                            string_itens = '- O item '
                            string_itens = string_itens + itens[0]
                            string_itens = string_itens + ' foi declarado ' + resultado.lower()
                        if string_itens != '':
                            p = doc.add_paragraph()
                            p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
                            r = p.add_run(f'{string_itens}')
                            r.bold = False
                            font = r.font
                            font.name = 'Times New Roman'
                            font.size = Pt(12)

doc.add_paragraph(f'\n{tomorrow}')
###############################################################################################################################################################

# SIGNATURES
assinaturas() # Source function at beggining of code
###############################################################################################################################################################

# PAGE HEADER
section = doc.sections[0]
header = section.header
for para in header.paragraphs:
  if '#K' in para.text:
    para.text = ''
    r = para.add_run(f'Proc. {str(processo)}')
    r.bold = True
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
###############################################################################################################################################################
doc.save(f"TERMO DE HOMOLOGAÇÃO - {titulo.replace('/','-')}.docx")